In [1]:
# Import the Pandas library

import openpyxl
import io
import tempfile
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from models.linear_regression import LinReg

lin_reg = LinReg()
path = "../.test_data/dataprocess.xlsx"

coded_df = lin_reg.process_excel(path)


print(coded_df.info())
coded_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024 entries, 0 to 3023
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NAME             3024 non-null   object 
 1   name_code        3024 non-null   int64  
 2   date             3024 non-null   int64  
 3   color_and_value  3024 non-null   object 
 4   value            3024 non-null   float64
 5   color            3024 non-null   object 
 6   color_code       3024 non-null   int64  
 7   next_color_code  3015 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 189.1+ KB
None


,NAME,name_code,date,color_and_value,value,color,color_code,next_color_code
0,GJ1,0,1,-16.155 | 00FFFF,-16.15500,00FFFF,0,3.0
1,GJ2,1,1,-10.131 | 00FFFF,-10.13100,00FFFF,0,2.0
2,GJ3,2,1,6.719 | 00FF00,6.71900,00FF00,1,3.0
3,GJ4,3,1,8.2 | 00FFFF,8.20000,00FFFF,0,3.0
4,GJ5,4,1,-7.444 | 00FF00,-7.44400,00FF00,1,1.0
...,...,...,...,...,...,...,...,...
3019,GJ5,4,336,-3.38613 | 8EA9DB,-3.38613,8EA9DB,8,NaN
3020,GJ6,5,336,-9.74303 | FFFF00,-9.74303,FFFF00,3,NaN
3021,GJ7,6,336,-10.9962 | A9D08E,-10.99620,A9D08E,5,NaN
3022,GJ8,7,336,16.04017 | FFC000,16.04017,FFC000,7,NaN


In [2]:
lin_reg.color_mapping



{0: '00FFFF',
 1: '00FF00',
 2: 'FF9900',
 3: 'FFFF00',
 4: 'D5A6BD',
 5: 'A9D08E',
 6: '9BC2E6',
 7: 'FFC000',
 8: '8EA9DB',
 9: 'FF0000'}

In [3]:
lin_reg.last_df

,NAME,2024-04-16 00:00:00
0,GJ1,-19.8579 | FFFF00
1,GJ2,-10.1783 | FFFF00
2,GJ3,-7.68448 | FFFF00
3,GJ4,-12.2896 | 8EA9DB
4,GJ5,-3.38613 | 8EA9DB
5,GJ6,-9.74303 | FFFF00
6,GJ7,-10.9962 | A9D08E
7,GJ8,16.04017 | FFC000
8,GJ9,-5.71004 | 8EA9DB


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = lin_reg.train_test_split(coded_df)


print(X_train.shape)
print(y_train.shape)

(2412, 4)
(2412, 10)


In [5]:
# Create and train the model


lin_reg.fit(X_train, X_test, y_train, y_test)

print(f'mse : {lin_reg.mse}' )

print(f'accuracy : {lin_reg.accuracy}' )
print(f'preferred accuracy: {lin_reg.preferred_accuracy}' )

mse : 4.39966832504146
accuracy : 0.28524046434494194
preferred accuracy: 1.0


In [6]:
lin_reg.last_df

,NAME,2024-04-16 00:00:00
0,GJ1,-19.8579 | FFFF00
1,GJ2,-10.1783 | FFFF00
2,GJ3,-7.68448 | FFFF00
3,GJ4,-12.2896 | 8EA9DB
4,GJ5,-3.38613 | 8EA9DB
5,GJ6,-9.74303 | FFFF00
6,GJ7,-10.9962 | A9D08E
7,GJ8,16.04017 | FFC000
8,GJ9,-5.71004 | 8EA9DB


In [7]:
# make predictions
lin_reg.predict()


,NAME,2024-04-16 00:00:00,next_color_code,next_color
0,GJ1,-19.8579 | FFFF00,8,8EA9DB
1,GJ2,-10.1783 | FFFF00,8,8EA9DB
2,GJ3,-7.68448 | FFFF00,8,8EA9DB
3,GJ4,-12.2896 | 8EA9DB,8,8EA9DB
4,GJ5,-3.38613 | 8EA9DB,8,8EA9DB
5,GJ6,-9.74303 | FFFF00,8,8EA9DB
6,GJ7,-10.9962 | A9D08E,8,8EA9DB
7,GJ8,16.04017 | FFC000,8,8EA9DB
8,GJ9,-5.71004 | 8EA9DB,8,8EA9DB
